In [4]:
import numpy as np
from scipy import ndimage, fft, stats
from matplotlib import pyplot as plt
from matplotlib.pyplot import imread

In [6]:
PLOT_RESULTS = True
RADIUS = 30
STD = 5
DPI=120

## 1.1 Fourier transformation

In [8]:
# ----- load the input image
img = imread('inputs/tuebingen.jpg')[:, :, 1] / 255.  
img_size = img.shape[0]


# TODO
FT_img_log = ...


# ----- plot input image together with fourier transformation
if PLOT_RESULTS:   #plot_results:
    plt.figure(figsize=(12, 12),dpi=DPI)
    plt.subplot(121)
    plt.xticks([]), plt.yticks([])
    plt.imshow(img, cmap = 'gray')
    plt.title('Input image')

    plt.subplot(122)
    plt.xticks([]), plt.yticks([])
    plt.imshow(FT_img_log, cmap='gray')
    plt.title('Fourier transformation')
    plt.show()

## 1.2 High-pass filter in spectral space

In [9]:
# TODO
def highpass_filter(img_size, radius):
    ''' Returns a highpass filter.
    input:
        img_size: length of the filter
        redius: radius of the circular aperture
    output:
        highpass: a filter of size(img_size, img_size) with values zero and one
    '''
    highpass = np.zeros((img_size, img_size))
    highpass = ...
    return highpass

# TODO
#   step 2: apply filter (with radius 30) to the spectrum
highpass = highpass_filter(img_size, radius=RADIUS) 
img_highpass = ...


# TODO
#   step 3: transform the spectrum back to image space                     
img_filtered = ...

if PLOT_RESULTS:   #plot_results:
    plt.figure(figsize=(12, 12),dpi=DPI)
    plt.subplot(121)
    plt.xticks([]), plt.yticks([])
    plt.imshow(np.abs(img_highpass)**0.15, cmap = 'gray')   # Potenz dient der Kontrastanpassung
    plt.title('Highpass filter applied in fourier space')

    plt.subplot(122)
    plt.xticks([]), plt.yticks([])
    plt.imshow((img_filtered), cmap=plt.cm.gray)
    plt.title('Backtransformation of the highpass filtered image')
    plt.show()

## 1.3 Gaussian filter

### 1.3.1 Construct a Gaussian filter

In [5]:
# TODO
def g_kern(kernlen=5, std=1):
    ''' Returns a 2D Gaussian kernel with standard devitation sig
    input:
        kernlen: length of the filter
        std: standard deviation of the gaussian in pixels
    output:
        g_kern_2d: a gaussian kernel of size (kernlen, kernlen) with sum 1
    '''
    g_kern_2d = ...
    return g_kern_2d

### 1.3.2 Apply Gaussian filter in image space

In [8]:
# TODO
#   step 2: create such a gaussian filter with standard deviation 3 and filtersize 25 and
#           convolve the image with this filter 
gaussfilter_img = ...
img_conv = ...

### 1.3.3 Transform the Gaussian filter to fourier space

In [1]:
# TODO
#   step 3: transform the gaussian kernel to fourier space
gaussfilter_f = ...
gaussfilter_f_shift = ...

### 1.3.4 Apply this filter in fourier space

In [10]:
f_gauss = ...
img_gauss_f = ...

### 1.3.5. Visuallise and compare two filtering methods

In [15]:
#   step 5: compare the results
difference = np.mean(np.abs(img_gauss_f - img_conv))       
print('difference: ' + str(difference))

if PLOT_RESULTS:
    # compare the two images
    plt.figure(figsize=(12, 12),dpi=DPI)

    plt.subplot(221)        # plt.subplot(122)
    plt.xticks([]), plt.yticks([])
    plt.imshow((gaussfilter_img), cmap=plt.cm.gray)
    plt.title('Gaussian (image space)')

    plt.subplot(222)        # plt.subplot(121)
    plt.xticks([]), plt.yticks([])
    plt.imshow(np.abs(gaussfilter_f_shift), cmap=plt.cm.gray)
    plt.title('Gaussian (fourier space)')

    plt.subplot(223)        # plt.subplot(122)
    plt.xticks([]), plt.yticks([])
    plt.imshow((img_conv), cmap=plt.cm.gray)      # img_conv
    plt.title('Gaussian filter applied in image space')

    plt.subplot(224)        # plt.subplot(121)
    plt.xticks([]), plt.yticks([])
    plt.imshow((img_gauss_f), cmap=plt.cm.gray)
    plt.title('Gaussian filter applied in fourier space')

    plt.show()

### 1.3.6 Splitted convolutional filter

In [ ]:
# TODO
def g_kern1d(kernlen=5, std=1):
    '''
    Returns a 1D Gaussian kernel with standard devitation sig 
    input:
        kernlen: length of the filter std: standard deviation of the gaussian in  pixels 
    output: 
        g_kern_1d: a gaussian kernel of size (kernlen, 1)
    '''
    g_kern_1d = ... 
    return g_kern_1d

gaussfilter_img_1D = ...
img_conv_sep_final = ...

# ----- plot results
if PLOT_RESULTS:   #plot_results:
    plt.figure(figsize=(12, 12),dpi=DPI)
    plt.subplot(121)
    plt.xticks([]), plt.yticks([])
    plt.imshow((img_conv), cmap=plt.cm.gray)      
    plt.title('Gaussian filter (2D) applied in image space')
    plt.subplot(122)
    plt.xticks([]), plt.yticks([])
    plt.imshow(img_conv_sep_final, cmap='gray')
    plt.title('Gaussian filter (separated) applied in image space')
    plt.show()